In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np

In [3]:
# Hyper-parameters

input_size = 32 * 32 * 3 #DA NON CAMBIARE
hidden_size = [80] #[50] 
num_classes = 10 #DA NON CAMBIARE
num_epochs =  10 #10 
learning_rate = 0.002 #1e-3
learning_rate_decay =  0.95 #0.95
reg=0.0001 #0.001 
batch_size = 400 

num_training= 49000 # DA NON CAMBIARE
num_validation =1000 # DA NON CAMBIARE
train = True

In [4]:
# Load the CIFAR-10 dataset
#-------------------------------------------------
norm_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                     ])
cifar_dataset = torchvision.datasets.CIFAR10(root='datasets/',
                                           train=True,
                                           transform=norm_transform,
                                           download=True)

test_dataset = torchvision.datasets.CIFAR10(root='datasets/',
                                          train=False,
                                          transform=norm_transform
                                          )
#-------------------------------------------------
# Prepare the training and validation splits
#-------------------------------------------------
mask = list(range(num_training))
train_dataset = torch.utils.data.Subset(cifar_dataset, mask)
mask = list(range(num_training, num_training + num_validation))
val_dataset = torch.utils.data.Subset(cifar_dataset, mask)

#-------------------------------------------------
# Data loader
#-------------------------------------------------
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                           batch_size=batch_size,
                                           shuffle=False)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

Extracting datasets/cifar-10-python.tar.gz to datasets/


In [6]:
def weights_init(m):
    if type(m) == nn.Linear:
        m.weight.data.normal_(0.0, 1e-3)
        m.bias.data.fill_(0.)

def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [7]:
# Device configuration

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device: %s'%device)


Using device: cuda


In [8]:
class MultiLayerPerceptron(nn.Module):
    def __init__(self, input_size, hidden_layers, num_classes):
        super(MultiLayerPerceptron, self).__init__()
        #################################################################################
        # TODO: Initialize the modules required to implement the mlp with the layer     #
        # configuration. input_size --> hidden_layers[0] --> hidden_layers[1] .... -->  #
        # hidden_layers[-1] --> num_classes                                             #
        # Make use of linear and relu layers from the torch.nn module                   #
        #################################################################################
        
        layers = [] #Use the layers list to store a variable number of layers
        
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        l1 = nn.Sequential(
            nn.Linear(input_size, hidden_layers[0]),
            nn.ReLU()
        )
        l2 = nn.Sequential(
            nn.Linear(hidden_layers[0],num_classes),
            #nn.ReLU() #CATE TI ODIA #e finalmente ha un motivo per farlo
        )

        layers.append(l1)
        layers.append(l2)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        # Enter the layers into nn.Sequential, so the model may "see" them
        # Note the use of * in front of layers
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        #################################################################################
        # TODO: Implement the forward pass computations                                  #
        # Note that you do not need to use the softmax operation at the end.             #
        # Softmax is only required for the loss computation and the criterion used below #
        # nn.CrossEntropyLoss() already integrates the softmax and the log loss together #
        #################################################################################
        
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        out = []
        for img in x:
            res = model.layers(img.flatten()) 
            out.append(res)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        
        return out

model = MultiLayerPerceptron(input_size, hidden_size, num_classes).to(device)

In [ ]:
#model.forward(train_loader)

In [9]:
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())



Model's state_dict:
layers.0.0.weight 	 torch.Size([80, 3072])
layers.0.0.bias 	 torch.Size([80])
layers.1.0.weight 	 torch.Size([10, 80])
layers.1.0.bias 	 torch.Size([10])


In [12]:
if train:
    model.apply(weights_init)
    model.train() #set dropout and batch normalization layers to training mode

    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=reg)

    # Train the model
    lr = learning_rate
    total_step = len(train_loader)
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):
            # Move tensors to the configured device
            images = images.to(device)
            labels = labels.to(device)
            #################################################################################
            # TODO: Implement the training code                                             #
            # 1. Pass the images to the model                                               #
            # 2. Compute the loss using the output and the labels.                          #
            # 3. Compute gradients and update the model using the optimizer                 #
            # Use examples in https://pytorch.org/tutorials/beginner/pytorch_with_examples.html
            #################################################################################
            # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

            scores = model.forward(images)
            scores = torch.stack(scores)
            loss = criterion(scores, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

            if (i+1) % 100 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                       .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

        # Code to update the lr
        lr *= learning_rate_decay
        update_lr(optimizer, lr)
        with torch.no_grad():
            correct = 0
            total = 0
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels.to(device)
                ####################################################
                # TODO: Implement the evaluation code              #
                # 1. Pass the images to the model                  #
                # 2. Get the most confident predicted class        #
                ####################################################
                # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
                
                scores = model.forward(images)
                scores = torch.stack(scores)
                
                scores = torch.softmax(scores,dim=1)
                predicted = torch.max(scores, 1)[1]

                # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            print('Validataion accuracy is: {} %'.format(100 * correct / total))

    ##################################################################################
    # TODO: Now that you can train a simple two-layer MLP using above code, you can  #
    # easily experiment with adding more layers and different layer configurations   #
    # and let the pytorch library handle computing the gradients                     #
    #                                                                                #
    # Experiment with different number of layers (at least from 2 to 5 layers) and   #
    # record the final validation accuracies Report your observations on how adding  #
    # more layers to the MLP affects its behavior. Try to improve the model          #
    # configuration using the validation performance as the guidance. You can        #
    # experiment with different activation layers available in torch.nn, adding      #
    # dropout layers, if you are interested. Use the best model on the validation    #
    # set, to evaluate the performance on the test set once and report it            #
    ##################################################################################

    # Save the model checkpoint
    torch.save(model.state_dict(), 'model.ckpt')

else:
    # Run the test code once you have your by setting train flag to false
    # and loading the best model

    best_model = None
    best_model = torch.load('model.ckpt')
    
    model.load_state_dict(best_model)
    
    # Test the model
    model.eval() #set dropout and batch normalization layers to evaluation mode
    
    # In test phase, we don't need to compute gradients (for memory efficiency)
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            ####################################################
            # TODO: Implement the evaluation code              #
            # 1. Pass the images to the model                  #
            # 2. Get the most confident predicted class        #
            ####################################################
            # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

            predicted = []
            scores = model.forward(images)
            scores = torch.stack(scores)
            predicted = torch.max(scores, 1)[1] #!!!!!!

            # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            if total == 1000:
                break

        print('Accuracy of the network on the {} test images: {} %'.format(total, 100 * correct / total))

Epoch [1/10], Step [100/123], Loss: 1.6593
Validataion accuracy is: 48.0 %
Epoch [2/10], Step [100/123], Loss: 1.5826
Validataion accuracy is: 48.3 %
Epoch [3/10], Step [100/123], Loss: 1.4298
Validataion accuracy is: 49.1 %
Epoch [4/10], Step [100/123], Loss: 1.3417
Validataion accuracy is: 50.6 %
Epoch [5/10], Step [100/123], Loss: 1.4495
Validataion accuracy is: 52.2 %
Epoch [6/10], Step [100/123], Loss: 1.3068
Validataion accuracy is: 50.0 %
Epoch [7/10], Step [100/123], Loss: 1.1518
Validataion accuracy is: 52.1 %
Epoch [8/10], Step [100/123], Loss: 1.2460
Validataion accuracy is: 52.7 %
Epoch [9/10], Step [100/123], Loss: 1.2195
Validataion accuracy is: 50.8 %
Epoch [10/10], Step [100/123], Loss: 1.1914
Validataion accuracy is: 52.6 %


In [15]:
best_model = None
best_model = torch.load('model.ckpt')

model.load_state_dict(best_model)

# Test the model
model.eval() #set dropout and batch normalization layers to evaluation mode

# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        ####################################################
        # TODO: Implement the evaluation code              #
        # 1. Pass the images to the model                  #
        # 2. Get the most confident predicted class        #
        ####################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        predicted = []
        scores = model.forward(images)
        scores = torch.stack(scores)
        predicted = torch.max(scores, 1)[1] #!!!!!!

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if total == 1000:
            break
    print('Accuracy of the network on the {} test images: {} %'.format(total, 100 * correct / total))

Accuracy of the network on the 10000 test images: 51.39 %
